In [16]:
import os
pid = os.fork()

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/lok419/Desktop/JupyterLab/Trading/venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/lok419/Desktop/JupyterLab/Trading/venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/lok419/Desktop/JupyterLab/Trading/venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/Users/lok419/Desktop/JupyterLab/Trading/venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/Library/Framewo

In [14]:
%load_ext autoreload
%autoreload 2

import numpy as np
from utils.data_helper import *
from utils.data import *
from utils.db import duck
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from strategy_v4.Data.Data import DataLayer
from strategy_v4.Model.run import *
from strategy_v4.Model.LinearModels import *
from strategy_v4.Evaluate.Evaluate import Evaluate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
instruments = [
    'META',
    'TSLA',
    'NVDA',
    'AAPL',    
    'DXJ',
    'BRK-B',
    'SPY',
    'QQQ',
    'NANC',
    'BTC',    
]

In [4]:
# dl_args = {key: value for key, value in vars(DATA_LAYER).items() if not key.startswith('_')}
# data = DataLayer(**dl_args)
# data.load()
# data.process()
#data.upload()
#df = data.df

In [5]:
dl_args = {key: value for key, value in vars(DATA_LAYER).items() if not key.startswith('_')}
df = DataLayer(**dl_args).get()
df.columns

2025-02-03 03:07:15,514 - Data Layer (sp500) - INFO - start_date: 2017-01-01
2025-02-03 03:07:15,515 - Data Layer (sp500) - INFO - end_date: 2024-01-29
2025-02-03 03:07:15,515 - Data Layer (sp500) - INFO - getting data files data/parquet/model_sp500_20170101_20240129.parquet, last updated at 2025-02-02 20:51:02


Index(['date', 'asset', 'close', 'high', 'low', 'open', 'volume', 'return10d',
       'return1d', 'return20d', 'return3d', 'return5d', 'return60d', 'rsi14d',
       'rsi28d', 'rsi2d', 'rsi60d', 'std10d', 'std20d', 'std5d', 'std60d',
       'volume10d', 'volume1d', 'volume20d', 'volume3d', 'volume5d',
       'volume60d'],
      dtype='object')

# Try to use price to generate more non-linear combination of features

In [6]:
df_pred, models = run_once('Lasso', instruments)
#df_pred, models = run_this(RidgeReg(), assets=['AAPL', 'TSLA'], features=['Return5d', 'Return1d', 'Return10d'])

2025-02-03 00:00:45,156 - Data Layer (sp500) - INFO - start_date: 2017-01-01
2025-02-03 00:00:45,157 - Data Layer (sp500) - INFO - end_date: 2024-01-29
2025-02-03 00:00:45,158 - Data Layer (sp500) - INFO - getting data files data/parquet/model_sp500_20170101_20240129.parquet, last updated at 2025-02-02 20:51:02
 80%|████████  | 8/10 [00:01<00:00,  8.97it/s]

skipped DXJ
skipped SPY
skipped QQQ
skipped NANC


100%|██████████| 10/10 [00:01<00:00,  7.30it/s]

skipped BTC


In [6]:
# eval = Evaluate(asset_level=False)
# eval.load()
# eval.eval()
# eval.df_eval

2025-02-03 03:10:51,827 - Evaluate - INFO - loaded Lasso.
2025-02-03 03:10:51,934 - Evaluate - INFO - loaded Ridge.
2025-02-03 03:10:52,121 - Evaluate - INFO - loaded LinearRegression.
2025-02-03 03:10:52,237 - Evaluate - INFO - loaded ElasticNet.


,pred_type,model,r2,mse,mae,accuracy_score,up_recall,up_precision,up_f1_score,down_recall,down_precision,down_f1_score
0,in-sample,ElasticNet,0.022575,0.000094,0.006403,0.548669,0.703697,0.575944,0.633443,0.355541,0.490559,0.412277
1,in-sample,Lasso,0.022545,0.000094,0.006403,0.548754,0.704952,0.575860,0.633900,0.354170,0.490652,0.411387
2,in-sample,LinearRegression,0.098286,0.000086,0.006310,0.568742,0.632276,0.606353,0.619043,0.489660,0.516793,0.502861
3,in-sample,Ridge,0.074215,0.000089,0.006326,0.568803,0.643350,0.604203,0.623162,0.476118,0.517472,0.495935
4,out-sample,ElasticNet,-0.026385,0.000075,0.006262,0.517595,0.635752,0.542330,0.585337,0.381358,0.475873,0.423405
5,out-sample,Lasso,-0.026157,0.000075,0.006261,0.517957,0.638077,0.542474,0.586404,0.379456,0.476223,0.422367
6,out-sample,LinearRegression,-0.202784,0.000088,0.006828,0.497542,0.458499,0.536132,0.494286,0.542577,0.464932,0.500763
7,out-sample,Ridge,-0.173240,0.000085,0.006762,0.500910,0.459650,0.539995,0.496594,0.548501,0.468165,0.505159


In [7]:
db = duck("return_predictions")
db.list_tables()

,name
0,model_eval
1,model_pred


In [12]:
db.query("""--sql         
    select         
        model
        , r2
        , mse
        , mae
        , accuracy_score
        , up_recall
        , up_precision
        , up_f1_score
        , down_recall
        , down_precision
        , down_f1_score        
        , time         
    from model_eval
    where 1=1
    and pred_type = 'out-sample'
    and time = (select max(time) from model_eval)
    order by model
""")

,model,r2,mse,mae,accuracy_score,up_recall,up_precision,up_f1_score,down_recall,down_precision,down_f1_score,time
0,ElasticNet,-0.027369,0.000073,0.006201,0.516436,0.628131,0.541981,0.581885,0.387571,0.474609,0.426697,2025-02-03 03:30:59.940518
1,Lasso,-0.027225,0.000073,0.006201,0.516489,0.628804,0.541978,0.582172,0.386908,0.474634,0.426305,2025-02-03 03:30:59.940518
2,LinearRegression,-0.179876,0.000084,0.006751,0.496197,0.450402,0.535379,0.489228,0.549033,0.464054,0.502979,2025-02-03 03:30:59.940518
3,Ridge,-0.159113,0.000082,0.006695,0.500541,0.453668,0.540273,0.493198,0.554619,0.468056,0.507674,2025-02-03 03:30:59.940518


In [27]:
# db.query("""--sql         
#     select    
#         model
#         , asset
#         , r2
#         , mse
#         , mae
#         , accuracy_score
#         , up_recall
#         , up_precision
#         , up_f1_score
#         , down_recall
#         , down_precision
#         , down_f1_score     
#         , time                  
#     from model_eval_asset
#     where 1=1
#     and pred_type = 'out-sample'
#     and time = (select max(time) from model_eval_asset)
#     order by asset
# """)